# Demonstration of SMARTpy's API

This notebook showcases an example of the use of the API of SMARTpy. It uses the input files available in the folder `examples/in/ExampleDaily`. The output files that should be generated following this tutorial are made available for comparison purposes in `examples/out/ExampleDaily`.

## Preamble

To reduce the number of paths the user needs to provide, SMARTpy requires the creation by the user of a directory (called `root` hereafter) containing two subdirectories named `in` and `out`. Within the subdirectory `in`, a subsubdirectory having the same name as the catchment to be modelled must be created: this is in this directory that SMARTpy will be looking for the input files. Following this setup, all the catchments the user desires to model can be gathered in one place, and it makes things much easier when dealing with large samples.

The structure of the root directory should follow the pattern described in the following schematic (where catchment can be replaced by the actual name of the catchment):

    root/
        in/
            catchment/
                catchment.rain
                catchment.peva
                catchment.flow
                catchment.parameters
        out/

## Setting up

After having installed `smartpy` (following the instructions given in the README file), the first thing that needs to be done is, of course, to import the smartpy package to start using its modules, classes, and functions.

In [ ]:
import smartpy

Then, you need to create an instance of the class SMART for the catchment you desire to model. By doing so, all the input data will be collected from the input files. All arguments are mandatory except `gauged_area_m2`. If `gauged_area_m2` is provided, it means that discharge measurements are available, and SMARTpy is expecting a *.flow* file in the input folder. These measurements can be used later on in post-processing to evaluate the performance of the simulation.

In [ ]:
from datetime import datetime, timedelta

sm = smartpy.SMART(
    catchment='ExampleDaily', 
    catchment_area_m2=175.46, 
    start=datetime.strptime('01/01/2007 09:00:00', '%d/%m/%Y %H:%M:%S'), 
    end=datetime.strptime('31/12/2016 09:00:00', '%d/%m/%Y %H:%M:%S'), 
    time_delta_simu=timedelta(hours=1), 
    time_delta_report=timedelta(days=1), 
    warm_up_days=365, 
    in_format='csv', 
    root='examples/',
    gauged_area_m2=175.97
)

Now it is time to give the model its parameter values. The recommended way to go is to use a file containing the parameter values, following the template in the example file used for this tutorial. The given values will be accessible in the attribute `{SMART_instance}.parameters.values` (where SMART_instance is sm in this tutorial, that it is to say the name binding to the instance of SMART used).

In [ ]:
sm.set_parameters_with_file('examples/in/ExampleDaily/ExampleDaily.parameters')

Alternatively, you can avoid the creation of a *.parameters* file and directly provide them as a dictionary, as follows:

In [ ]:
sm.set_parameters_with_dict(
    {
        'T': 1.0,
        'C': 1.0,
        'H': 0.20845,
        'D': 0.24606,
        'S': 0.0001230,
        'Z': 105.26,
        'SK': 46.82,
        'FK': 315.55,
        'GK': 1066.73,
        'RK': 10.64
    }
)

## Simulating

The SMART instance has now all it needs to start the simulation for the catchment over the time period set up. One last instruction and the simulations will be under way. The method simulate needs to be given the parameter values, they are directly accessible from the SMART instance attributes.

In [ ]:
sm.simulate(sm.parameters.values)

## Writing output files

First, one might want to save the simulation results in output files. This can be done using the following instruction:

In [ ]:
sm.write_output_files()

## Post-processing

SMART comes with a few objective functions that can be used to assess the performance of the simulation against evaluation data. These objectives functions are only usable if the `gauged_area_m2` was provided in the SMART object instance. If this is the case, the *.flow* has already been read and the data is directly available from the SMART instance (see below). In a similar way, the SMART instance gives you access (after simulation) to the simulated discharge. You can then evaluate these two series using one of the objective functions available in smartpy.objfunctions (for example the Nash Sutcliffe Efficiency here).

In [ ]:
evaluation_ = smart_model.get_evaluation_array()
simulation_ = smart_model.get_simulation_array()

The evaluation and simulation arrays returned have the same length. However, it is frequent to have days with missing discharge measurements, that would have been absent from the file. SMARTpy assigned a `nan` (not a number) value for these days with missing data, this is why the two arrays have the same length. However, when comparing them using an objective function, only days with available observations can be used. To deal with this, a solution is as follows:

In [ ]:
import numpy as np

evaluation = evaluation_[~np.isnan(evaluation_)]
simulation = simulation_[~np.isnan(evaluation_)]

Now that the arrays still have the same length but with no missing value, the objective function can be calculated.

In [ ]:
nse = smartpy.objfunctions.nash_sutcliffe(evaluation=evaluation, simulation=simulation)